In [1]:
%load_ext autoreload
%autoreload 2

from sanity_check.traffic_generation import *

In [3]:
scenarios: List[Scenario] = []
instances: Dict[Scenario, List[TrafficGenerator]] = {}
instance_paths: List[str] = glob(os.path.join("/home/jovyan/work/binary_files/debug/ids/", "*"))

In [15]:
def parse_instances() :
    for instance_path in instance_paths:
        scenario = parse_instance_parameters(os.path.join(instance_path, "parameters.py"))
        scenarios.append(scenario)

        node_paths = glob(os.path.join(instance_path, "*.conf_traffic_generator_run.log"))
        generators: List[TrafficGenerator] = []
        for node_path in node_paths:
            node_generator = parse_traffic_generator(node_path)
            generators.append(node_generator)

        instances[scenario] = generators

parse_instances()

In [25]:
def run_stats():
    epidemic_should = 0
    epidemic_sent = 0
    epidemic_failed_run = 0
    epidemic_ok_run = 0
    epidemic_really_sent = 0

    cadr_should = 0
    cadr_sent = 0
    cadr_failed_run = 0
    cadr_ok_run = 0
    cadr_really_sent = 0
    
    for scenario in scenarios:
        for generator in instances[scenario]:
            if scenario.routing == "epidemic":
                epidemic_should = epidemic_should + generator.num_dtnd
                epidemic_sent = epidemic_sent + generator.num_sent
                epidemic_really_sent = epidemic_really_sent + generator.really_sent
                if generator.num_sent != generator.num_dtnd:
                    epidemic_failed_run = epidemic_failed_run + 1
                else:
                    epidemic_ok_run = epidemic_ok_run + 1
            else:
                cadr_should = cadr_should + generator.num_dtnd
                cadr_sent = cadr_sent + generator.num_sent
                cadr_really_sent = cadr_really_sent + generator.really_sent
                if generator.num_sent != generator.num_dtnd:
                    cadr_failed_run = cadr_failed_run + 1
                else:
                    cadr_ok_run = cadr_ok_run + 1

    print(f"Epidemic")
    print(f"  Generator should send {epidemic_sent} bundles, but actually sent {epidemic_really_sent} bundles.")
    print(f"  DTND received {epidemic_should} bundles, {epidemic_sent - epidemic_should}) missing.")
    print(f"  {epidemic_ok_run} nodes successful, {epidemic_failed_run} failed ({(epidemic_failed_run/(epidemic_failed_run + epidemic_ok_run)) * 100}%)")
    
    print(f"CADR")
    print(f"  Generator should send {cadr_sent} bundles, but actually sent {cadr_really_sent} bundles.")
    print(f"  DTND received {cadr_should} bundles, {cadr_sent - cadr_should}) missing.")
    print(f"  {cadr_ok_run} nodes successful, {cadr_failed_run} failed ({(cadr_failed_run/(cadr_failed_run + cadr_ok_run)) * 100}%)")
run_stats()

Epidemic
  Generator should send 128450 bundles, but actually sent 75248 bundles.
  DTND received 75246 bundles, 53204) missing.
  1129 nodes successful, 1006 failed (47.11943793911007%)
CADR
  Generator should send 128100 bundles, but actually sent 113532 bundles.
  DTND received 113534 bundles, 14566) missing.
  1354 nodes successful, 746 failed (35.523809523809526%)


In [28]:
for scenario in scenarios:
    for other_scenario in scenarios:
        if scenario.instance_id == other_scenario.instance_id:
            continue
        if scenario.matches(other_scenario):
            generators_a = instances[scenario]
            generators_b = instances[other_scenario]

            for generator in generators_a:
                if generator.num_sent != generator.really_sent:
                    print(f"Mismatch: {scenario.instance_id}/{generator.node_id}({scenario.routing})")
                    print(f"  {generator.num_sent}, {generator.really_sent}")
                    if generator.really_sent != generator.num_dtnd:
                        print(f"    {generator.num_dtnd}")
                
                for other_generator in generators_b:
                    if generator == other_generator:
                        if not generator.matches(other_generator):
                            foo = generator.matches(other_generator)
                            print(
                                f"#### FAIL: Mismatch between instances {scenario.instance_id} and {other_scenario.instance_id}:"
                            )
                            print(
                                f"#### FAIL: Params: bundles_per_node: {scenario.bundles_per_node}, seed: {scenario.seed}, payload_size: {scenario.payload_size}"
                            )
                            print(f"#### FAIL: Node: {generator.node_id}")
                            print("#### FAIL: Differences:")
                            generator.difference(other_generator)
                            print("#### FAIL: -----------------------------")

Mismatch: 670/n48(epidemic)
  10, 9
Mismatch: 670/n41(epidemic)
  10, 8
Mismatch: 670/n49(epidemic)
  10, 8
Mismatch: 670/n20(epidemic)
  10, 9
Mismatch: 670/n43(epidemic)
  10, 8
Mismatch: 670/n32(epidemic)
  10, 8
Mismatch: 670/n36(epidemic)
  10, 7
Mismatch: 670/n42(epidemic)
  10, 9
Mismatch: 670/n48(epidemic)
  10, 9
Mismatch: 670/n41(epidemic)
  10, 8
Mismatch: 670/n49(epidemic)
  10, 8
Mismatch: 670/n20(epidemic)
  10, 9
Mismatch: 670/n43(epidemic)
  10, 8
Mismatch: 670/n32(epidemic)
  10, 8
Mismatch: 670/n36(epidemic)
  10, 7
Mismatch: 670/n42(epidemic)
  10, 9
Mismatch: 670/n48(epidemic)
  10, 9
Mismatch: 670/n41(epidemic)
  10, 8
Mismatch: 670/n49(epidemic)
  10, 8
Mismatch: 670/n20(epidemic)
  10, 9
Mismatch: 670/n43(epidemic)
  10, 8
Mismatch: 670/n32(epidemic)
  10, 8
Mismatch: 670/n36(epidemic)
  10, 7
Mismatch: 670/n42(epidemic)
  10, 9
Mismatch: 670/n48(epidemic)
  10, 9
Mismatch: 670/n41(epidemic)
  10, 8
Mismatch: 670/n49(epidemic)
  10, 8
Mismatch: 670/n20(epidemic)


In [ ]:
Epidemic:
    generator sent 256900 (150496) bundles, DTND received 150492 (missing 106408). Total OK runs: 2258, total failed runs: 2012 (ratio: 0.4711943793911007)
CADR: generator sent 256900 (227764) bundles, DTND received 227768 (missing 29132). Total OK runs: 2778, total failed runs: 1492 (ratio: 0.34941451990632316)